In [1]:
import os, joblib
import numpy as np
import pandas as pd
from spark.preprocessor import load_q_data, load_timeseries_data, train_test_split_ids, Preprocess_Q
from spark.model_io import load_model
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from scipy.stats import randint, uniform


from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder


## Load Data

In [3]:
#Data file directory
raw_prep_dir  = '../processed_data/'

In [4]:
#Load questionair data
X_q_data, y_data, id_list = load_q_data(raw_prep_dir + 'merged_dfq.csv')

In [5]:
X_q_data

,id,age,age_at_diagnosis,bmi,height,weight,gender,handedness,appearance_in_kinship,01,...,21,22,23,24,25,26,27,28,29,30
0,1,56,56,26.061679,173,78,male,right,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,81,69,27.920213,193,104,male,right,False,True,...,True,True,True,False,True,False,True,False,True,False
2,3,45,45,26.989619,170,78,female,right,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,67,63,34.720883,161,90,female,right,False,False,...,True,True,True,False,False,True,True,True,False,False
4,5,75,65,29.069767,172,86,male,left,False,True,...,False,True,True,True,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,465,65,62,26.122449,175,80,male,right,True,True,...,False,False,False,False,True,False,False,False,False,False
465,466,84,84,25.013521,172,74,female,right,True,False,...,False,False,False,False,False,False,False,False,False,False
466,467,57,55,27.700831,190,100,male,right,False,False,...,False,False,True,False,False,False,False,False,False,False
467,468,76,73,30.098969,198,118,male,right,False,False,...,True,False,False,False,True,True,True,True,False,False


In [6]:
X_train_id, X_test_id, y_train, y_test = train_test_split_ids (id_list, y_data, test_size = 0.2, stratify = True)
print(X_train_id.shape, X_test_id.shape)

(375,) (94,)


In [7]:
X_train_q = X_q_data[X_q_data.id.isin(X_train_id)]
X_test_q = X_q_data[X_q_data.id.isin(X_test_id)]
print(X_train_q.shape)

(375, 39)


In [8]:
X_train_q

,id,age,age_at_diagnosis,bmi,height,weight,gender,handedness,appearance_in_kinship,01,...,21,22,23,24,25,26,27,28,29,30
1,2,81,69,27.920213,193,104,male,right,False,True,...,True,True,True,False,True,False,True,False,True,False
4,5,75,65,29.069767,172,86,male,left,False,True,...,False,True,True,True,True,True,True,False,False,False
5,6,72,60,39.328340,171,115,female,right,False,False,...,False,False,True,False,False,True,True,True,False,False
6,7,74,73,28.692653,181,94,male,right,False,True,...,True,False,True,False,False,True,True,False,True,False
7,8,73,65,23.030045,168,65,female,right,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,465,65,62,26.122449,175,80,male,right,True,True,...,False,False,False,False,True,False,False,False,False,False
465,466,84,84,25.013521,172,74,female,right,True,False,...,False,False,False,False,False,False,False,False,False,False
466,467,57,55,27.700831,190,100,male,right,False,False,...,False,False,True,False,False,False,False,False,False,False
467,468,76,73,30.098969,198,118,male,right,False,False,...,True,False,False,False,True,True,True,True,False,False


In [9]:
#check labels for y_train
display((y_data[y_data["id"].isin(X_train_id)]["label"] - y_train).sum())
#check labels for y_test
display((y_data[y_data["id"].isin(X_test_id)]["label"] - y_test).sum())

np.int64(0)

np.int64(0)

## Preprocess Data

In [10]:
prep = Preprocess_Q(feature_importance = False)
prep.fit(X_train_q)
X_q_train_prep = prep.transform(X_train_q)
X_q_test_prep = prep.transform(X_test_q)
pd.DataFrame(X_q_train_prep)

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.588235,1.0000,1.333333,0.857143,0.060221,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.352941,0.6250,-0.222222,0.122449,0.066685,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
2,0.058824,0.4375,-0.296296,1.306122,0.124373,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.823529,0.5625,0.444444,0.448980,0.064565,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,0.352941,0.5000,-0.518519,-0.734694,0.032722,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,0.176471,0.0000,0.000000,-0.122449,0.050112,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
371,1.470588,1.1875,-0.222222,-0.367347,0.043876,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372,-0.235294,-0.5000,1.111111,0.693878,0.058987,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
373,0.823529,0.6875,1.703704,1.428571,0.072473,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0


In [11]:
check = pd.DataFrame(X_q_train_prep)
check.describe().T

,count,mean,std,min,25%,50%,75%,max
0,375.0,-0.086431,0.771843,-3.294118,-0.529412,0.000000,0.470588,1.705882
1,375.0,-0.046667,0.659928,-1.562500,-0.562500,0.000000,0.437500,1.437500
2,375.0,-0.084741,0.842454,-8.888889,-0.555556,0.000000,0.444444,2.074074
3,375.0,-0.010449,0.742526,-1.551020,-0.551020,0.000000,0.448980,4.000000
4,375.0,0.058556,0.056554,0.000000,0.038127,0.050982,0.068565,1.000000
5,375.0,0.589333,0.492612,0.000000,0.000000,1.000000,1.000000,1.000000
6,375.0,0.936000,0.245080,0.000000,1.000000,1.000000,1.000000,1.000000
7,375.0,0.277333,0.448281,0.000000,0.000000,0.000000,1.000000,1.000000
8,375.0,0.272000,0.445584,0.000000,0.000000,0.000000,1.000000,1.000000
9,375.0,0.352000,0.478232,0.000000,0.000000,0.000000,1.000000,1.000000


## Load Model

In [12]:
#model = load_model("/Users/sebastian.zilles/code/marzecovaa/spark/model/model_20250908-155122.joblib")

## Model

In [ ]:
#base models
base_models = [
    ('svc', SVC(probability=True, class_weight='balanced', random_state=42)),
    ('catboost', CatBoostClassifier(loss_function='MultiClass',eval_metric='MultiClass',auto_class_weights='Balanced', thread_count=-1,random_seed=42,verbose=0)),
     ]

#define meta model
meta_model = LogisticRegression(
    class_weight='balanced',
    C=0.5,
    max_iter=10000,
    solver='lbfgs',
    multi_class='auto',
    random_state=42
)

#stacking classifier
stacked_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    stack_method='predict_proba',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    passthrough=True,
    n_jobs=-1
)

## Model Fit & Scoring

In [45]:
model = stacked_model.fit(X_q_train_prep, y_train)

/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: invalid val

In [47]:
y_pred = model.predict(X_q_test_prep)
balanced_accuracy_score(y_test, y_pred)

/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


ValueError: Found input variables with inconsistent numbers of samples: [94, 375]

In [42]:
prep_q = Preprocess_Q(feature_importance=False).fit(X_train_q)
X_prep = prep_q.transform(X_train_q)

import numpy as np
variances = X_prep.var(axis=0)
print("Zero-variance features idx:", np.where(variances == 0)[0])


Zero-variance features idx: []


In [43]:
# After transform
X_prep = prep_q.transform(X_train_q)

import numpy as np
print("Has NaN:", np.isnan(X_prep).any())
print("Has Inf:", np.isinf(X_prep).any())

# Where are the offenders?
bad_cols_nan = np.where(np.isnan(X_prep).any(axis=0))[0]
bad_cols_inf = np.where(np.isinf(X_prep).any(axis=0))[0]
print("NaN cols:", bad_cols_nan)
print("Inf cols:", bad_cols_inf)

Has NaN: False
Has Inf: False
NaN cols: []
Inf cols: []


In [29]:
cv_scores = cross_validate(
    model, X_q_train_prep, y_train,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=False),
    scoring='balanced_accuracy'
)

/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: invalid val

In [30]:
score = np.mean(cv_scores["test_score"])
score

np.float64(0.3240875693507273)